In [1]:
import random
import pandas as pd 
from copy import deepcopy
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, median_absolute_error,mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import pickle
# from skopt import BayesSearchCV
import datetime as dt
import itertools

import warnings
warnings.filterwarnings('ignore')

In [2]:
data_path = "/Users/jurajkapasny/Data/energy_hack/"
df = pd.read_csv(data_path+"spotreba_prepared.csv",sep = ";")

In [3]:
df["timestamp"] = df["Dátum a čas"] + " " + df["Unnamed: 1"]
df = df[["spotreba","om","timestamp"]]
df.head()

,spotreba,om,timestamp
0,0.211,1,2016-01-01 00:15:00
1,0.210,1,2016-01-01 00:30:00
2,0.210,1,2016-01-01 00:45:00
3,0.206,1,2016-01-01 01:00:00
4,0.205,1,2016-01-01 01:15:00


In [4]:
om_info = pd.read_csv(data_path+"om_info_prepared.csv",sep = ";")
om_info.drop("Unnamed: 0",axis = 1, inplace=True)

In [51]:
om_info.head()

,Číslo OM,Zapojenie,Druh tarify,Inštalovaný výkon FVE (kWp)
0,1,A,D2,2.0
1,2,A,D2,7.2
2,3,A,D2,2.0
3,4,A,D1,0.0
4,5,C,D4,9.2


#### Adding tarif to main_df 

In [5]:
df_final = df.merge(om_info[["Číslo OM",
                             "Druh tarify",
                             "Inštalovaný výkon FVE (kWp)"]], 
                    how = "left", 
                    left_on = "om", 
                    right_on = "Číslo OM")

In [6]:
df_final.head()

,spotreba,om,timestamp,Číslo OM,Druh tarify,Inštalovaný výkon FVE (kWp)
0,0.211,1,2016-01-01 00:15:00,1,D2,2.0
1,0.210,1,2016-01-01 00:30:00,1,D2,2.0
2,0.210,1,2016-01-01 00:45:00,1,D2,2.0
3,0.206,1,2016-01-01 01:00:00,1,D2,2.0
4,0.205,1,2016-01-01 01:15:00,1,D2,2.0


#### Adding vyroba

In [7]:
vyroba = pd.read_csv(data_path+"vyroba_prepared.csv",sep = ";")
# vyroba.drop("Unnamed: 0",axis = 1, inplace=True)

In [8]:
vyroba_tr = pd.DataFrame(vyroba.set_index("time").stack()).reset_index()
vyroba_tr.tail()

,time,level_1,0
245947,2016-12-31 23:44:59.993,3.15,0.0
245948,2016-12-31 23:44:59.993,4,0.0
245949,2016-12-31 23:44:59.993,5,0.0
245950,2016-12-31 23:44:59.993,7.2,0.0
245951,2016-12-31 23:44:59.993,9.2,0.0


In [9]:
vyroba_tr.time = pd.to_datetime(vyroba_tr.time).dt.round('1s')
vyroba_tr.level_1 = vyroba_tr.level_1.astype(float)
df_final.timestamp = pd.to_datetime(df_final.timestamp)

In [10]:
vyroba_tr.head()

,time,level_1,0
0,2016-01-01,2.00,0.0
1,2016-01-01,2.50,0.0
2,2016-01-01,3.15,0.0
3,2016-01-01,4.00,0.0
4,2016-01-01,5.00,0.0


In [11]:
df_final = df_final.merge(vyroba_tr[["time","level_1",0]], 
                          how="left",
                          left_on = ["timestamp","Inštalovaný výkon FVE (kWp)"],
                          right_on = ["time","level_1"])

In [12]:
df_final.drop(["time","level_1","Číslo OM"],axis = 1, inplace=True)

In [13]:
df_final.count()

spotreba                       5245996
om                             5245996
timestamp                      5245996
Druh tarify                    5245996
Inštalovaný výkon FVE (kWp)    5245996
0                              3489396
dtype: int64

In [14]:
df_final.fillna(0,inplace = True)

In [15]:
# df_final.head()
df_final = df_final.rename({0: "vyroba", 
                            "Inštalovaný výkon FVE (kWp)":"vykon",
                           "Druh tarify":"tarifa"}, axis=1)

In [16]:
len(df_final.om.unique())

150

#### getting the diff for everyone everytime

In [17]:
df_final["diff"] = df_final.vyroba - df_final.spotreba

# total_spotreba_per_day = df_with_day[["day","spotreba","vyroba"]].groupby("day")[["spotreba","vyroba"]].sum()

In [18]:
df_final.count()

spotreba     5245996
om           5245996
timestamp    5245996
tarifa       5245996
vykon        5245996
vyroba       5245996
diff         5245996
dtype: int64

In [19]:
df_final.head()

,spotreba,om,timestamp,tarifa,vykon,vyroba,diff
0,0.211,1,2016-01-01 00:15:00,D2,2.0,0.0,-0.211
1,0.210,1,2016-01-01 00:30:00,D2,2.0,0.0,-0.210
2,0.210,1,2016-01-01 00:45:00,D2,2.0,0.0,-0.210
3,0.206,1,2016-01-01 01:00:00,D2,2.0,0.0,-0.206
4,0.205,1,2016-01-01 01:15:00,D2,2.0,0.0,-0.205


#### percentages

In [20]:
all_times = list(df_final.timestamp.unique())
df_final["viac_spotreby"] = 0
df_final.loc[df_final["diff"] <= 0, "viac_spotreby"] = 1

In [21]:
temp_gr = df_final.groupby(["timestamp","viac_spotreby","om"])[["diff"]].sum()

In [22]:
temp_gr_pcts = temp_gr.groupby(level=[0,1]).apply(lambda x:
                                                 100 * x / float(x.sum()))

In [23]:
temp_gr_pcts = temp_gr_pcts.rename({"diff": "percentage"}, axis=1)
temp_gr_pcts = temp_gr_pcts.reset_index()

In [24]:
df_final = df_final.merge(temp_gr_pcts.reset_index(), how = "left", on = ["timestamp","om"])

In [25]:
df_final.head()

,spotreba,om,timestamp,tarifa,vykon,vyroba,diff,viac_spotreby_x,index,viac_spotreby_y,percentage
0,0.211,1,2016-01-01 00:15:00,D2,2.0,0.0,-0.211,1,147,1,0.697163
1,0.210,1,2016-01-01 00:30:00,D2,2.0,0.0,-0.210,1,294,1,0.696575
2,0.210,1,2016-01-01 00:45:00,D2,2.0,0.0,-0.210,1,441,1,0.696899
3,0.206,1,2016-01-01 01:00:00,D2,2.0,0.0,-0.206,1,588,1,0.668940
4,0.205,1,2016-01-01 01:15:00,D2,2.0,0.0,-0.205,1,735,1,0.692088


In [26]:
df_final = df_final.rename({"viac_spotreby_x": "viac_spotreby"}, axis=1)
df_final.drop("viac_spotreby_y", axis = 1, inplace=True)

In [27]:
# total_energy_left = df_final[["timestamp","diff"]].groupby("timestamp").sum("diff")

In [34]:
# only those with profit 
# df_with_day_profit = temp[temp["diff"] > 0]
# df_with_day_loss = temp[temp["diff"] <= 0]
# df_with_day_loss["percentage"] = df_with_day_loss["diff"] / df_with_day_loss["diff"].sum()
# df_with_day_profit["percentage"] = df_with_day_profit["diff"] / df_with_day_profit["diff"].sum()

# total_energy_left = temp[temp["diff"] > 0]["diff"].sum()
# total_energy_needed = -temp[temp["diff"] <= 0]["diff"].sum()
total_energy_left = df_final[["timestamp","viac_spotreby","diff"]].groupby(["timestamp","viac_spotreby"]).sum()
index = pd.MultiIndex.from_product([all_times, [0,1]], names = ["timestamp", "viac_spotreby"])
all_comb = pd.DataFrame(index = index).reset_index()

total_energy_left = all_comb.merge(total_energy_left.reset_index(), how = "left", on = ["timestamp","viac_spotreby"])

total_energy_left.fillna(0, inplace=True)

# bad_guys = list(temp[temp["diff"] < 0].om.unique())

In [35]:
total_energy_left.head()

,timestamp,viac_spotreby,diff
0,2016-01-01 00:15:00,0,0.0000
1,2016-01-01 00:15:00,1,-30.2655
2,2016-01-01 00:30:00,0,0.0000
3,2016-01-01 00:30:00,1,-30.1475
4,2016-01-01 00:45:00,0,0.0000


In [36]:
total_energy_left["diff_abs"] = total_energy_left["diff"].abs()
temp_max = total_energy_left.groupby(["timestamp"])[["diff_abs"]].max()
temp_max["ind"] = "value"
temp_max = temp_max.reset_index()

temp_max = total_energy_left.merge(temp_max,
                       how = "left",
                                  on = ["timestamp","diff_abs"])

temp_max = temp_max[temp_max.ind.notnull()][["timestamp","viac_spotreby"]]
temp_max.rename({"viac_spotreby":"viac_spotreby_total"},axis = 1, inplace=True)
temp_max.head()

,timestamp,viac_spotreby_total
1,2016-01-01 00:15:00,1
3,2016-01-01 00:30:00,1
5,2016-01-01 00:45:00,1
7,2016-01-01 01:00:00,1
9,2016-01-01 01:15:00,1


In [37]:
total_energy_left.rename({"diff": "diff_total"},
                                            axis =1,
                                            inplace = True)

In [39]:
df_final = df_final.merge(total_energy_left[["timestamp","viac_spotreby","diff_total"]], how = "left", on = ["timestamp","viac_spotreby"])

In [40]:
df_final.head()

,spotreba,om,timestamp,tarifa,vykon,vyroba,diff,viac_spotreby,index,percentage,diff_total
0,0.211,1,2016-01-01 00:15:00,D2,2.0,0.0,-0.211,1,147,0.697163,-30.2655
1,0.210,1,2016-01-01 00:30:00,D2,2.0,0.0,-0.210,1,294,0.696575,-30.1475
2,0.210,1,2016-01-01 00:45:00,D2,2.0,0.0,-0.210,1,441,0.696899,-30.1335
3,0.206,1,2016-01-01 01:00:00,D2,2.0,0.0,-0.206,1,588,0.668940,-30.7950
4,0.205,1,2016-01-01 01:15:00,D2,2.0,0.0,-0.205,1,735,0.692088,-29.6205


In [309]:
res = df_final.copy(deep=True)

In [310]:
res = res.merge(temp_max, on = "timestamp")

#### what to buy and sell to community

In [311]:
res = res.merge(total_energy_left[["timestamp","viac_spotreby","diff_total"]].set_index(["timestamp","viac_spotreby"]).unstack().reset_index(),
                how="left",
                on = "timestamp"
               )

In [312]:
res.head()

,spotreba,om,timestamp,tarifa,vykon,vyroba,diff,viac_spotreby,index,percentage,diff_total,viac_spotreby_total,"(diff_total, 0)","(diff_total, 1)"
0,0.211,1,2016-01-01 00:15:00,D2,2.0,0.0,-0.211,1,147,0.697163,-30.2655,1,0.0,-30.2655
1,0.060,2,2016-01-01 00:15:00,D2,7.2,0.0,-0.060,1,148,0.198246,-30.2655,1,0.0,-30.2655
2,0.292,3,2016-01-01 00:15:00,D2,2.0,0.0,-0.292,1,149,0.964795,-30.2655,1,0.0,-30.2655
3,1.535,4,2016-01-01 00:15:00,D1,0.0,0.0,-1.535,1,150,5.071781,-30.2655,1,0.0,-30.2655
4,0.460,5,2016-01-01 00:15:00,D4,9.2,0.0,-0.460,1,151,1.519882,-30.2655,1,0.0,-30.2655


In [313]:
res.rename({res.columns[-2]:"total_produced"}, axis = 1, inplace = True)
res.rename({res.columns[-1]:"total_spent"}, axis = 1, inplace = True)

In [314]:
res["nakup_from_community"] = 0
res.loc[(res.viac_spotreby == 1) & (res.viac_spotreby_total == 1), "nakup_from_community"] = \
        (res.loc[(res.viac_spotreby == 1) & (res.viac_spotreby_total == 1)].percentage / 100 ) * \
         res.loc[(res.viac_spotreby == 1) & (res.viac_spotreby_total == 1)].total_produced
        

res.loc[(res.viac_spotreby == 1) & (res.viac_spotreby_total == 0), "nakup_from_community"] = \
        -res.loc[(res.viac_spotreby == 1) & (res.viac_spotreby_total == 0)]["diff"]


res["nakup_from_network"] = 0
res.loc[res.viac_spotreby == 1, "nakup_from_network"] = \
- res.loc[res.viac_spotreby == 1]["diff"] - res.loc[res.viac_spotreby == 1]["nakup_from_community"]

    
res["predaj_to_community"] = 0
res.loc[(res.viac_spotreby == 0) & (res.viac_spotreby_total == 0), "predaj_to_community"] = \
        (res.loc[(res.viac_spotreby == 0) & (res.viac_spotreby_total == 0)].percentage / 100 ) * \
         - res.loc[(res.viac_spotreby == 0) & (res.viac_spotreby_total == 0)].total_spent

res.loc[(res.viac_spotreby == 0) & (res.viac_spotreby_total == 1), "predaj_to_community"] = \
        res.loc[(res.viac_spotreby == 0) & (res.viac_spotreby_total == 1)]["diff"]

    
res["predaj_to_network"] = 0
res.loc[res.viac_spotreby == 0, "predaj_to_network"] = \
         res.loc[res.viac_spotreby == 0]["diff"] - res.loc[res.viac_spotreby == 0]["predaj_to_community"]

In [315]:
res = res.drop("index", axis = 1)

In [316]:
res.head()

,spotreba,om,timestamp,tarifa,vykon,vyroba,diff,viac_spotreby,percentage,diff_total,viac_spotreby_total,total_produced,total_spent,nakup_from_community,nakup_from_network,predaj_to_community,predaj_to_network
0,0.211,1,2016-01-01 00:15:00,D2,2.0,0.0,-0.211,1,0.697163,-30.2655,1,0.0,-30.2655,0.0,0.211,0.0,0.0
1,0.060,2,2016-01-01 00:15:00,D2,7.2,0.0,-0.060,1,0.198246,-30.2655,1,0.0,-30.2655,0.0,0.060,0.0,0.0
2,0.292,3,2016-01-01 00:15:00,D2,2.0,0.0,-0.292,1,0.964795,-30.2655,1,0.0,-30.2655,0.0,0.292,0.0,0.0
3,1.535,4,2016-01-01 00:15:00,D1,0.0,0.0,-1.535,1,5.071781,-30.2655,1,0.0,-30.2655,0.0,1.535,0.0,0.0
4,0.460,5,2016-01-01 00:15:00,D4,9.2,0.0,-0.460,1,1.519882,-30.2655,1,0.0,-30.2655,0.0,0.460,0.0,0.0


In [211]:
# res.to_csv(data_path + "data_prepared_for_dashboard.csv", sep = ";", index = False)

#### Add Prices

In [317]:
price_zo_siete = pd.read_csv(data_path+"price_zo_siete.csv",sep = ";")

In [318]:
price_zo_siete.head()

,druh_tarify,hodiny,price
0,D1,0,0.119989
1,D1,1,0.119989
2,D1,2,0.119989
3,D1,3,0.119989
4,D1,4,0.119989


In [319]:
res["hour"] = res.timestamp.dt.hour

In [320]:
res = res.merge(price_zo_siete, how = "left",left_on=["tarifa","hour"],right_on=["druh_tarify","hodiny"])

In [321]:
res.rename({"price":"price_zo_siete"},axis = 1, inplace=True)

In [322]:
price_z_komunity = pd.read_csv(data_path+"price_z_komunity.csv",sep = ";")
add_on_price_z_komunity = pd.read_csv(data_path + "add_on_price_z_komunity.csv", sep = ";")

In [323]:
price_z_komunity.timestamp = pd.to_datetime(price_z_komunity.timestamp).dt.round('1s')

In [324]:
price_z_komunity.tail()

,timestamp,main_price
35131,2016-12-31 22:45:00,0.057197
35132,2016-12-31 23:00:00,0.055030
35133,2016-12-31 23:15:00,0.013757
35134,2016-12-31 23:30:00,0.027515
35135,2016-12-31 23:45:00,0.041272


In [325]:
len(price_do_komunity)

35136

In [326]:
add_on_price_z_komunity

,tarifa,cena
0,D1,0.048533
1,D2,0.032936
2,D3,0.032936
3,D4,0.027581
4,D5,0.027581


In [327]:
res = res.merge(price_z_komunity, how = "left", on = "timestamp")
res = res.merge(add_on_price_z_komunity, how = "left", on = "tarifa")

In [328]:
res["price_z_komunity"] = res["main_price"] + res["cena"]
res.drop(["main_price","cena"],axis = 1, inplace=True)

In [329]:
price_do_komunity = pd.read_csv(data_path + "price_do_komunity.csv", sep = ";")
price_do_komunity.head()

,timestamp,main_price
0,2016-01-01 00:00:00.000,0.02490
1,2016-01-01 00:15:00.000,0.02501
2,2016-01-01 00:30:00.000,0.02512
3,2016-01-01 00:45:00.000,0.02523
4,2016-01-01 01:00:00.000,0.02534


In [330]:
price_do_komunity.timestamp = pd.to_datetime(price_do_komunity.timestamp).dt.round('1s')

In [331]:
res = res.merge(price_do_komunity, how = "left", on = "timestamp")
res.rename({"main_price":"price_do_komunity"},axis = 1, inplace=True)

In [332]:
price_do_siete = pd.read_csv(data_path + "price_do_siete.csv", sep = ";")
price_do_siete.head()

,timestamp,main_price
0,2016-01-01 00:00:00.000,0.02090
1,2016-01-01 00:15:00.000,0.02101
2,2016-01-01 00:30:00.000,0.02112
3,2016-01-01 00:45:00.000,0.02123
4,2016-01-01 01:00:00.000,0.02134


In [333]:
price_do_siete.timestamp = pd.to_datetime(price_do_siete.timestamp).dt.round('1s')

In [334]:
res = res.merge(price_do_komunity, how = "left", on = "timestamp")
res.rename({"main_price":"price_do_siete"},axis = 1, inplace=True)

In [335]:
res.columns

Index(['spotreba', 'om', 'timestamp', 'tarifa', 'vykon', 'vyroba', 'diff',
       'viac_spotreby', 'percentage', 'diff_total', 'viac_spotreby_total',
       'total_produced', 'total_spent', 'nakup_from_community',
       'nakup_from_network', 'predaj_to_community', 'predaj_to_network',
       'hour', 'druh_tarify', 'hodiny', 'price_zo_siete', 'price_z_komunity',
       'price_do_komunity', 'price_do_siete'],
      dtype='object')

In [336]:
res.drop(["druh_tarify","hodiny","hour","vykon"],axis = 1, inplace=True)

In [337]:
res["income_from_komunity"] = res["predaj_to_community"] * res["price_do_komunity"]
res["income_from_network"] = res["predaj_to_network"] * res["price_do_siete"]
res["expense_to_komunity"] = res["nakup_from_community"] * res["price_z_komunity"]
res["expense_to_network"] = res["nakup_from_network"] * res["price_zo_siete"]

In [338]:
clusters = pd.read_csv(data_path + "clusters.csv", sep = ";")
res = res.merge(clusters, on = "om", how="left")

In [339]:
# res["cluster"] = res["tarifa"]

In [340]:
lows = []
highs = []
for value in list(res["cluster"].unique()):
    
    lows.append(res.loc[res.cluster == value][["spotreba","timestamp","cluster"]].groupby(["timestamp","cluster"])[["spotreba"]].quantile(0.1).reset_index())
    highs.append(res.loc[res.cluster == value][["spotreba","timestamp","cluster"]].groupby(["timestamp","cluster"])[["spotreba"]].quantile(0.9).reset_index())
    
#     res.loc[res.cluster == value,"low_border"] = res.loc[res.cluster == value].spotreba.quantile(0.2)
#     res.loc[res.cluster == value,"high_border"] = res.loc[res.cluster == value].spotreba.quantile(0.95)

In [341]:
for dataframe in lows:
    dataframe.columns = ["timestamp","cluster","lower_bound"]
for dataframe in highs:
    dataframe.columns = ["timestamp","cluster","upper_bound"]

In [342]:
res = res.merge(pd.concat(lows).reset_index(drop=True), how = "left", on = ["timestamp","cluster"])

In [343]:
res = res.merge(pd.concat(highs).reset_index(drop=True), how = "left", on = ["timestamp","cluster"])

In [344]:
res.count()

spotreba                5245996
om                      5245996
timestamp               5245996
tarifa                  5245996
vyroba                  5245996
diff                    5245996
viac_spotreby           5245996
percentage              5245996
diff_total              5245996
viac_spotreby_total     5245996
total_produced          5245996
total_spent             5245996
nakup_from_community    5245996
nakup_from_network      5245996
predaj_to_community     5245996
predaj_to_network       5245996
price_zo_siete          5245996
price_z_komunity        5245996
price_do_komunity       5245996
price_do_siete          5245996
income_from_komunity    5245996
income_from_network     5245996
expense_to_komunity     5245996
expense_to_network      5245996
cluster                 5245996
lower_bound             5245996
upper_bound             5245996
dtype: int64

In [345]:
# res.sort_values(["timestamp","om"])

In [346]:
res = res.sort_values(["om","timestamp"])

#### cost if vyroba a predaj je zakazany

In [347]:
res["cost_ak_vyroba_nejde"] = - res["spotreba"] * res["price_zo_siete"]
res["cost_ak_vyroba_ide"] = res["income_from_komunity"] + res["income_from_network"] - res["expense_to_komunity"] - res["expense_to_network"]

In [348]:
res.columns

Index(['spotreba', 'om', 'timestamp', 'tarifa', 'vyroba', 'diff',
       'viac_spotreby', 'percentage', 'diff_total', 'viac_spotreby_total',
       'total_produced', 'total_spent', 'nakup_from_community',
       'nakup_from_network', 'predaj_to_community', 'predaj_to_network',
       'price_zo_siete', 'price_z_komunity', 'price_do_komunity',
       'price_do_siete', 'income_from_komunity', 'income_from_network',
       'expense_to_komunity', 'expense_to_network', 'cluster', 'lower_bound',
       'upper_bound', 'cost_ak_vyroba_nejde', 'cost_ak_vyroba_ide'],
      dtype='object')

In [349]:
res['cost_ak_vyroba_nejde_1D'] = res.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_ak_vyroba_nejde'].rolling(96, min_periods=1).sum()).values
res['cost_ak_vyroba_nejde_2D'] = res.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_ak_vyroba_nejde'].rolling(96*2, min_periods=1).sum()).values
res['cost_ak_vyroba_nejde_3D'] = res.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_ak_vyroba_nejde'].rolling(96*3, min_periods=1).sum()).values
res['cost_ak_vyroba_nejde_4D'] = res.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_ak_vyroba_nejde'].rolling(96*4, min_periods=1).sum()).values
res['cost_ak_vyroba_nejde_5D'] = res.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_ak_vyroba_nejde'].rolling(96*5, min_periods=1).sum()).values
res['cost_ak_vyroba_nejde_6D'] = res.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_ak_vyroba_nejde'].rolling(96*6, min_periods=1).sum()).values
res['cost_ak_vyroba_nejde_7D'] = res.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_ak_vyroba_nejde'].rolling(96*7, min_periods=1).sum()).values

res['cost_ak_vyroba_ide_1D'] = res.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_ak_vyroba_ide'].rolling(96, min_periods=1).sum()).values
res['cost_ak_vyroba_ide_2D'] = res.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_ak_vyroba_ide'].rolling(96*2, min_periods=1).sum()).values
res['cost_ak_vyroba_ide_3D'] = res.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_ak_vyroba_ide'].rolling(96*3, min_periods=1).sum()).values
res['cost_ak_vyroba_ide_4D'] = res.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_ak_vyroba_ide'].rolling(96*4, min_periods=1).sum()).values
res['cost_ak_vyroba_ide_5D'] = res.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_ak_vyroba_ide'].rolling(96*5, min_periods=1).sum()).values
res['cost_ak_vyroba_ide_6D'] = res.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_ak_vyroba_ide'].rolling(96*6, min_periods=1).sum()).values
res['cost_ak_vyroba_ide_7D'] = res.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_ak_vyroba_ide'].rolling(96*7, min_periods=1).sum()).values


In [350]:
# res[res.cost_ak_vyroba_ide_7D > 0]

In [351]:
res['lower_quantile_1D'] = res.groupby('om').apply(lambda res: res.sort_values('timestamp')['spotreba'].rolling(96, min_periods=1).quantile(0.25)).values
res['upper_quantile_1D'] = res.groupby('om').apply(lambda res: res.sort_values('timestamp')['spotreba'].rolling(96, min_periods=1).quantile(0.75)).values
res['iqr_1D'] = res['upper_quantile_1D'] - res['lower_quantile_1D']

In [352]:
res.columns

Index(['spotreba', 'om', 'timestamp', 'tarifa', 'vyroba', 'diff',
       'viac_spotreby', 'percentage', 'diff_total', 'viac_spotreby_total',
       'total_produced', 'total_spent', 'nakup_from_community',
       'nakup_from_network', 'predaj_to_community', 'predaj_to_network',
       'price_zo_siete', 'price_z_komunity', 'price_do_komunity',
       'price_do_siete', 'income_from_komunity', 'income_from_network',
       'expense_to_komunity', 'expense_to_network', 'cluster', 'lower_bound',
       'upper_bound', 'cost_ak_vyroba_nejde', 'cost_ak_vyroba_ide',
       'cost_ak_vyroba_nejde_1D', 'cost_ak_vyroba_nejde_2D',
       'cost_ak_vyroba_nejde_3D', 'cost_ak_vyroba_nejde_4D',
       'cost_ak_vyroba_nejde_5D', 'cost_ak_vyroba_nejde_6D',
       'cost_ak_vyroba_nejde_7D', 'cost_ak_vyroba_ide_1D',
       'cost_ak_vyroba_ide_2D', 'cost_ak_vyroba_ide_3D',
       'cost_ak_vyroba_ide_4D', 'cost_ak_vyroba_ide_5D',
       'cost_ak_vyroba_ide_6D', 'cost_ak_vyroba_ide_7D', 'lower_quantile_1D',
    

#### Some final columns about the spendings (all of them 1 day aggregates) and for both variants, with creation and without
- cost OM za elekriku do siete
- cost OM za elektriku do komunity
- cost OM za distibuciu
- zisk OM z predaja do komunity
- zisk OM z predaja do komunity

- zisk zse total
- zisk zse z distribucie
- zisk zse z elektriky

In [394]:
res_cost = res[["timestamp","om",'price_zo_siete', 'tarifa', 'price_z_komunity', 'price_do_komunity',
       'price_do_siete', 'nakup_from_community',
       'nakup_from_network', 'predaj_to_community', 'predaj_to_network']]

In [395]:
cena_za_distribuciu_pri_nakupe_zo_siete = pd.DataFrame({"D1": 0.80889,"D2":0.054894,"D3":0.054894,"D4":0.045968,"D5":0.45968}, index=["cena"]).transpose()

In [396]:
cena_za_distribuciu_pri_nakupe_zo_siete = cena_za_distribuciu_pri_nakupe_zo_siete.reset_index()

In [397]:
cena_za_distribuciu_pri_nakupe_zo_siete.columns = ["tarifa","cena_za_distribuciu_pri_nakupe_zo_siete"]

In [398]:
res_cost = res_cost.merge(cena_za_distribuciu_pri_nakupe_zo_siete, how = "left", on = "tarifa")

In [399]:
# obchodovanie povolene
res_cost["cost_OM_za_el_zo_siete"] = res_cost["nakup_from_network"] * (res_cost["price_zo_siete"] - res_cost["cena_za_distribuciu_pri_nakupe_zo_siete"])
# obchodovanie zakazane
res_cost["cost_OM_za_el_zo_siete_no_power"] = (res_cost["nakup_from_community"] + res_cost["nakup_from_network"]) * (res_cost["price_zo_siete"] - res_cost["cena_za_distribuciu_pri_nakupe_zo_siete"])



In [400]:
cena_za_distribuciu_pri_nakupe_z_komunity = pd.DataFrame({"D1":0.0485334,
"D2":0.0329364,
"D3":0.0329364,
"D4":0.0275808,
"D5":0.0275808}, index = ["cena"]).transpose()
cena_za_distribuciu_pri_nakupe_z_komunity = cena_za_distribuciu_pri_nakupe_z_komunity.reset_index()
cena_za_distribuciu_pri_nakupe_z_komunity.columns = ["tarifa","cena_za_distribuciu_pri_nakupe_z_komunity"]

In [401]:
res_cost = res_cost.merge(cena_za_distribuciu_pri_nakupe_z_komunity, how = "left", on = "tarifa")

In [402]:
# obchodovanie povolene
res_cost["cost_OM_za_el_z_komunity"] = res_cost["nakup_from_community"] * (res_cost["price_z_komunity"] - res_cost["cena_za_distribuciu_pri_nakupe_z_komunity"])
# obchodovanie zakazane
res_cost["cost_OM_za_el_z_komunity_no_power"] = 0

In [403]:
# obchodovanie povolene
res_cost["cost_OM_za_distribuciu"] = (res_cost["nakup_from_community"] * res_cost["cena_za_distribuciu_pri_nakupe_z_komunity"]) + \
                                     (res_cost["nakup_from_network"] * res_cost["cena_za_distribuciu_pri_nakupe_zo_siete"])
# obchodovanie zakazane
res_cost["cost_OM_za_distribuciu_no_power"] = ((res_cost["nakup_from_network"] + res_cost["nakup_from_community"]) * res_cost["cena_za_distribuciu_pri_nakupe_zo_siete"])

In [404]:
# obchodovanie povolene
res_cost["revenue_OM_za_predaj_do_komunuty"] = res_cost["predaj_to_community"] * res_cost["price_do_komunity"]
res_cost["revenue_OM_za_predaj_do_siete"] = res_cost["predaj_to_network"] * res_cost["price_do_siete"]
# obchodovanie zakazane
res_cost["revenue_OM_za_predaj_do_komunuty_no_power"] = 0
res_cost["revenue_OM_za_predaj_do_siete_no_power"] = (res_cost["predaj_to_network"]+res_cost["predaj_to_community"]) * res_cost["price_do_siete"]

In [405]:
# obchodovanie povolene
res_cost["revenue_zse_total"] = (res_cost["nakup_from_network"] * res_cost["price_zo_siete"]) + \
                             (res_cost["nakup_from_community"] * res_cost["cena_za_distribuciu_pri_nakupe_z_komunity"]) + \
                                (res_cost["predaj_to_community"] * 0.008) - \
                                (res_cost["predaj_to_network"] * res_cost["price_do_siete"])
# obchodovanie zakazane
res_cost["revenue_zse_total_no_power"] = ((res_cost["nakup_from_network"]+res_cost["nakup_from_community"]) * res_cost["price_zo_siete"]) + \
                                ((res_cost["predaj_to_network"]+res_cost["predaj_to_community"]) * res_cost["price_do_siete"])

In [406]:
# obchodovanie povolene
res_cost["revenue_zse_distribucia"] = (res_cost["nakup_from_network"] * res_cost["cena_za_distribuciu_pri_nakupe_zo_siete"]) + \
                                      (res_cost["nakup_from_community"] * res_cost["cena_za_distribuciu_pri_nakupe_z_komunity"]) + \
                                      (res_cost["predaj_to_community"] * 0.008)
# obchodovanie zakazane
res_cost["revenue_zse_distribucia_no_power"] = ((res_cost["nakup_from_network"]+res_cost["nakup_from_community"]) * res_cost["cena_za_distribuciu_pri_nakupe_zo_siete"])

In [407]:
# obchodovanie povolene
res_cost["revenue_zse_elektrika"] = res_cost["revenue_zse_total"] - res_cost["revenue_zse_distribucia"]
# obchodovanie zakazane
res_cost["revenue_zse_elektrika_no_power"] = res_cost["revenue_zse_total_no_power"] - res_cost["revenue_zse_distribucia_no_power"]

In [422]:
res_cost = res_cost.sort_values(["om","timestamp"])

In [425]:
# rolling variables, 1D
res_cost['cost_OM_za_el_zo_siete_1D'] = res_cost.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_OM_za_el_zo_siete'].rolling(96, min_periods=1).sum()).values
res_cost['cost_OM_za_el_zo_siete_no_power_1D'] = res_cost.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_OM_za_el_zo_siete_no_power'].rolling(96, min_periods=1).sum()).values
res_cost['cost_OM_za_el_z_komunity_1D'] = res_cost.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_OM_za_el_z_komunity'].rolling(96, min_periods=1).sum()).values
res_cost['cost_OM_za_el_z_komunity_no_power_1D'] = res_cost.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_OM_za_el_z_komunity_no_power'].rolling(96, min_periods=1).sum()).values
res_cost['cost_OM_za_distribuciu_1D'] = res_cost.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_OM_za_distribuciu'].rolling(96, min_periods=1).sum()).values
res_cost['cost_OM_za_distribuciu_no_power_1D'] = res_cost.groupby('om').apply(lambda res: res.sort_values('timestamp')['cost_OM_za_distribuciu_no_power'].rolling(96, min_periods=1).sum()).values
res_cost['revenue_OM_za_predaj_do_komunuty_1D'] = res_cost.groupby('om').apply(lambda res: res.sort_values('timestamp')['revenue_OM_za_predaj_do_komunuty'].rolling(96, min_periods=1).sum()).values
res_cost['revenue_OM_za_predaj_do_siete_1D'] = res_cost.groupby('om').apply(lambda res: res.sort_values('timestamp')['revenue_OM_za_predaj_do_siete'].rolling(96, min_periods=1).sum()).values
res_cost['revenue_OM_za_predaj_do_komunuty_no_power_1D'] = res_cost.groupby('om').apply(lambda res: res.sort_values('timestamp')['revenue_OM_za_predaj_do_komunuty_no_power'].rolling(96, min_periods=1).sum()).values
res_cost['revenue_OM_za_predaj_do_siete_no_power_1D'] = res_cost.groupby('om').apply(lambda res: res.sort_values('timestamp')['revenue_OM_za_predaj_do_siete_no_power'].rolling(96, min_periods=1).sum()).values
res_cost['revenue_zse_total_1D'] = res_cost.groupby('om').apply(lambda res: res.sort_values('timestamp')['revenue_zse_total'].rolling(96, min_periods=1).sum()).values
res_cost['revenue_zse_total_no_power_1D'] = res_cost.groupby('om').apply(lambda res: res.sort_values('timestamp')['revenue_zse_total_no_power'].rolling(96, min_periods=1).sum()).values
res_cost['revenue_zse_distribucia_1D'] = res_cost.groupby('om').apply(lambda res: res.sort_values('timestamp')['revenue_zse_distribucia'].rolling(96, min_periods=1).sum()).values
res_cost['revenue_zse_distribucia_no_power_1D'] = res_cost.groupby('om').apply(lambda res: res.sort_values('timestamp')['revenue_zse_distribucia_no_power'].rolling(96, min_periods=1).sum()).values
res_cost['revenue_zse_elektrika_1D'] = res_cost.groupby('om').apply(lambda res: res.sort_values('timestamp')['revenue_zse_elektrika'].rolling(96, min_periods=1).sum()).values
res_cost['revenue_zse_elektrika_no_power_1D'] = res_cost.groupby('om').apply(lambda res: res.sort_values('timestamp')['revenue_zse_elektrika_no_power'].rolling(96, min_periods=1).sum()).values



In [429]:
res_cost.columns

Index(['timestamp', 'om', 'price_zo_siete', 'tarifa', 'price_z_komunity',
       'price_do_komunity', 'price_do_siete', 'nakup_from_community',
       'nakup_from_network', 'predaj_to_community', 'predaj_to_network',
       'cena_za_distribuciu_pri_nakupe_zo_siete', 'cost_OM_za_el_zo_siete',
       'cost_OM_za_el_zo_siete_no_power',
       'cena_za_distribuciu_pri_nakupe_z_komunity', 'cost_OM_za_el_z_komunity',
       'cost_OM_za_el_z_komunity_no_power', 'cost_OM_za_distribuciu',
       'cost_OM_za_distribuciu_no_power', 'revenue_OM_za_predaj_do_komunuty',
       'revenue_OM_za_predaj_do_siete',
       'revenue_OM_za_predaj_do_komunuty_no_power',
       'revenue_OM_za_predaj_do_siete_no_power', 'revenue_zse_total',
       'revenue_zse_total_no_power', 'revenue_zse_distribucia',
       'revenue_zse_distribucia_no_power', 'revenue_zse_elektrika',
       'revenue_zse_elektrika_no_power', 'cost_OM_za_el_zo_siete_1D',
       'cost_OM_za_el_zo_siete_no_power_1D', 'cost_OM_za_el_z_komunity_1D

In [428]:
res.drop(['cost_ak_vyroba_nejde_1D', 'cost_ak_vyroba_nejde_2D',
       'cost_ak_vyroba_nejde_3D', 'cost_ak_vyroba_nejde_4D',
       'cost_ak_vyroba_nejde_5D', 'cost_ak_vyroba_nejde_6D',
       'cost_ak_vyroba_nejde_7D', 'cost_ak_vyroba_ide_1D',
       'cost_ak_vyroba_ide_2D', 'cost_ak_vyroba_ide_3D',
       'cost_ak_vyroba_ide_4D', 'cost_ak_vyroba_ide_5D',
       'cost_ak_vyroba_ide_6D', 'cost_ak_vyroba_ide_7D'],axis = 1, inplace=True)

In [430]:
res_cost.drop(['price_zo_siete', 'tarifa', 'price_z_komunity',
       'price_do_komunity', 'price_do_siete', 'nakup_from_community',
       'nakup_from_network', 'predaj_to_community', 'predaj_to_network',
       'cena_za_distribuciu_pri_nakupe_zo_siete'], axis = 1, inplace=True)

In [431]:
res = res.merge(res_cost, how = "left", on = ["om","timestamp"])

In [432]:
res.head()

,spotreba,om,timestamp,tarifa,vyroba,diff,viac_spotreby,percentage,diff_total,viac_spotreby_total,...,revenue_OM_za_predaj_do_komunuty_1D,revenue_OM_za_predaj_do_siete_1D,revenue_OM_za_predaj_do_komunuty_no_power_1D,revenue_OM_za_predaj_do_siete_no_power_1D,revenue_zse_total_1D,revenue_zse_total_no_power_1D,revenue_zse_distribucia_1D,revenue_zse_distribucia_no_power_1D,revenue_zse_elektrika_1D,revenue_zse_elektrika_no_power_1D
0,0.199,1,2016-01-01 00:00:00,D2,0.0,-0.199,1,0.742011,-26.8190,1,...,0.0,0.0,0.0,0.0,0.019919,0.019919,0.010924,0.010924,0.008995,0.008995
1,0.211,1,2016-01-01 00:15:00,D2,0.0,-0.211,1,0.697163,-30.2655,1,...,0.0,0.0,0.0,0.0,0.041039,0.041039,0.022507,0.022507,0.018532,0.018532
2,0.210,1,2016-01-01 00:30:00,D2,0.0,-0.210,1,0.696575,-30.1475,1,...,0.0,0.0,0.0,0.0,0.062058,0.062058,0.034034,0.034034,0.028024,0.028024
3,0.210,1,2016-01-01 00:45:00,D2,0.0,-0.210,1,0.696899,-30.1335,1,...,0.0,0.0,0.0,0.0,0.083078,0.083078,0.045562,0.045562,0.037516,0.037516
4,0.206,1,2016-01-01 01:00:00,D2,0.0,-0.206,1,0.668940,-30.7950,1,...,0.0,0.0,0.0,0.0,0.103697,0.103697,0.056870,0.056870,0.046827,0.046827


In [420]:
# res_cost[res_cost["revenue_OM_za_predaj_do_komunuty"] != res_cost["revenue_OM_za_predaj_do_siete_no_power"]]

In [437]:
res.cluster = res.cluster.astype(str) + "C"

In [438]:
res.cluster.value_counts()

2C    2025272
3C    1510676
0C    1112804
1C     597244
Name: cluster, dtype: int64

In [248]:
# res.to_csv(data_path + "data_prepared_for_dashboard.csv", sep = ";", index = False)

In [306]:
# res.rename({"cluster_y":"cluster"},axis = 1, inplace=True)

In [307]:
# res.cluster.value_counts()

2    2025272
3    1510676
0    1112804
1     597244
Name: cluster, dtype: int64

In [439]:
res[res.timestamp >= "2016-09-30 00:00:00"].to_csv(data_path + "data_prepared_for_dashboard.csv", sep = ";", index = False)

In [297]:
# res[(res.om == 5) & (res.timestamp.between("2016-10-28 00:00:00","2016-10-31 00:00:00"))][["diff","diff_total","percentage","viac_spotreby","viac_spotreby_total",
#                                                                                            "expense_to_komunity","expense_to_network",
#                                                                                           "nakup_from_community","nakup_from_network"
#                                                                                           ]]